# Embedding CRUD Notebook

Este codigo incluye la clase EmbeddingCRUD la cual contiene las funcionalidades CRUD y las importaciones necesarias para el funcionamiento de la misma.

In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np

class EmbeddingCRUD:
    def __init__(self):
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.df = pd.DataFrame(columns=['text', 'embedding'])

    def create(self, text):
        embedding = self.model.encode(text)
        self.df = pd.concat([self.df, pd.DataFrame({'text': [text], 'embedding': [embedding.tolist()]})], ignore_index=True)
        print(f"Se ha creado un nuevo embedding para el texto: '{text}'")

    def read(self, query):
        query_embedding = self.model.encode(query)
        self.df['similarity'] = self.df['embedding'].apply(lambda x: np.dot(x, query_embedding) / (np.linalg.norm(x) * np.linalg.norm(query_embedding)))
        print(f"Resultados de la búsqueda para la consulta '{query}':")
        return self.df.sort_values('similarity', ascending=False)
    
    def update(self, index, new_text):
        old_text = self.df.at[index, 'text']
        self.df.at[index, 'text'] = new_text
        self.df.at[index, 'embedding'] = self.model.encode(new_text).tolist()
        print(f"El texto en el índice {index} ha sido actualizado de '{old_text}' a '{new_text}'")

    def delete(self, index):
        deleted_text = self.df.at[index, 'text']
        self.df = self.df.drop(index)
        print(f"Se ha eliminado el texto '{deleted_text}' del índice {index}")

C:\Users\Solvo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Example execution file

Instancia de la clase con los metodos CRUD

In [2]:
crud = EmbeddingCRUD()


##### Crear algunos embeddings

In [3]:
crud.create("This is a sample sentence")
crud.create("Another sentence for testing")

Se ha creado un nuevo embedding para el texto: 'This is a sample sentence'
Se ha creado un nuevo embedding para el texto: 'Another sentence for testing'


##### Buscar embeddings similares a una query

In [4]:
results = crud.read("similar sentence")
print(results)

Resultados de la búsqueda para la consulta 'similar sentence':
                           text  \
0     This is a sample sentence   
1  Another sentence for testing   

                                           embedding  similarity  
0  [0.05404346436262131, 0.08422980457544327, 0.0...    0.605037  
1  [-0.006979598663747311, 0.0739775225520134, -0...    0.438689  


##### Actualizar un embedding

In [5]:
crud.update(0, "Updated sentence")

El texto en el índice 0 ha sido actualizado de 'This is a sample sentence' a 'Updated sentence'


##### Eliminar un embedding

In [6]:
crud.delete(1)

Se ha eliminado el texto 'Another sentence for testing' del índice 1
